<H1> Les jeux de données pour le projet </H1>

Dans ce notebook nous présentons les jeux de données utilisés pour le projet. Nous proposons également des fonctions pour permettre de pouvoir facilement les données.   

Il n'y a donc plus qu'à chercher les meilleurs modèles et à répondre aux questions de l'énoncé du projet.   

Bon courage !

ps : il y a trois jeux de données et ils sont très différents donc attention vous aurez peut être 3 modèles différents.

## Installation



Avant de commencer, il est nécessaire de déjà posséder dans son environnement toutes les librairies utiles. Dans la seconde cellule nous importons toutes les librairies qui seront utiles à ce notebook. Il se peut que, lorsque vous lanciez l'éxecution de cette cellule, une soit absente. Dans ce cas il est nécessaire de l'installer. Pour cela dans la cellule suivante utiliser la commande :  

*! pip install nom_librairie*  

**Attention :** il est fortement conseillé lorsque l'une des librairies doit être installer de relancer le kernel de votre notebook.

**Remarque :** même si toutes les librairies sont importées dès le début, les librairies utiles pour des fonctions présentées au cours de ce notebook sont ré-importées de manière à indiquer d'où elles viennent et ainsi faciliter la réutilisation de la fonction dans un autre projet.



In [ ]:
# utiliser cette cellule pour installer les librairies manquantes
# pour cela il suffit de taper dans cette cellule : !pip install nom_librairie_manquante
# d'exécuter la cellule et de relancer la cellule suivante pour voir si tout se passe bien
# recommencer tant que toutes les librairies ne sont pas installées ...

# sous Colab il faut déjà intégrer ces deux librairies

#!pip install umap-learn[plot]
#!pip install holoviews
#!pip install -U ipykernel

# eventuellement ne pas oublier de relancer le kernel du notebook
!pip install --upgrade tensorflow
!pip install optuna
!pip install opencv-python-headless

In [ ]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys
import copy
import random
from numpy import mean
from numpy import std


# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import plotly.graph_objs as go
import plotly.offline as py

from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# TensorFlow et keras
import tensorflow as tf
from keras import layers
from keras import models
from keras import optimizers
import tensorflow as tf
from keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from keras.models import load_model
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import os
from os import listdir
from os.path import isfile, join
import cv2
import glob
import csv
from statistics import mean, stdev
import optuna
from sklearn.metrics import classification_report
import tensorflow.keras.backend as K
from gc import collect
from keras.layers import Lambda
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
def plot_curves(histories):
    """
    Fonction pour afficher les courbes de loss et d'accuracy moyennées et écart-types à travers les k-folds.

    Paramètres :
    - histories (list) : Historique d'entraînement des différents plis K-folds.
    """

    # Initialisation des figures
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Extraction du nombre d'époques d'après l'un des historiques
    epochs = range(len(histories[0].history['loss']))

    # Calcul des moyennes et des écart-types pour chaque époque
    mean_loss = np.mean([history.history['loss'] for history in histories], axis=0)
    std_loss = np.std([history.history['loss'] for history in histories], axis=0)

    mean_val_loss = np.mean([history.history['val_loss'] for history in histories], axis=0)
    std_val_loss = np.std([history.history['val_loss'] for history in histories], axis=0)

    mean_accuracy = np.mean([history.history['accuracy'] for history in histories], axis=0)
    std_accuracy = np.std([history.history['accuracy'] for history in histories], axis=0)

    mean_val_accuracy = np.mean([history.history['val_accuracy'] for history in histories], axis=0)
    std_val_accuracy = np.std([history.history['val_accuracy'] for history in histories], axis=0)

    # Couleurs pour les courbes
    train_color = 'blue'
    val_color = 'orange'

    # Courbes de loss avec moyenne et écart-type
    ax1.plot(epochs, mean_loss, color=train_color, label='Train')
    ax1.fill_between(epochs, mean_loss - std_loss, mean_loss + std_loss, color=train_color, alpha=0.2)

    ax1.plot(epochs, mean_val_loss, color=val_color, label='Validation')
    ax1.fill_between(epochs, mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, color=val_color, alpha=0.2)

    # Courbes d'accuracy avec moyenne et écart-type
    ax2.plot(epochs, mean_accuracy, color=train_color, label='Train')
    ax2.fill_between(epochs, mean_accuracy - std_accuracy, mean_accuracy + std_accuracy, color=train_color, alpha=0.2)

    ax2.plot(epochs, mean_val_accuracy, color=val_color, label='Validation')
    ax2.fill_between(epochs, mean_val_accuracy - std_val_accuracy, mean_val_accuracy + std_val_accuracy, color=val_color, alpha=0.2)

    # Titres, labels et légendes
    ax1.set_title(f'Loss (k={len(histories)})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    ax2.set_title(f'Accuracy (k={len(histories)})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.show()

def plot_loss_accuracy(history):
    """
    Fonction pour afficher les courbes de loss et d'accuracy pour une seule exécution.

    Paramètres :
    - history (History) : Historique d'entraînement retourné par model.fit().
    """

    # Extraction des données d'historique
    epochs = range(len(history.history['loss']))

    # Calcul des moyennes pour chaque époque
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # Initialisation des figures
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Courbes de loss
    ax1.plot(epochs, loss, color='blue', label='Train Loss')
    ax1.plot(epochs, val_loss, color='orange', label='Validation Loss')
    ax1.set_title('Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Courbes d'accuracy
    ax2.plot(epochs, accuracy, color='blue', label='Train Accuracy')
    ax2.plot(epochs, val_accuracy, color='orange', label='Validation Accuracy')
    ax2.set_title('Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    # Affichage des courbes
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(cm, classes, title='Matrice de confusion', cmap=plt.cm.Blues):
    """
    Affiche la matrice de confusion.

    Parameters:
    - cm (array-like): Matrice de confusion (2D numpy array).
    - classes (list of str): Liste des noms des classes correspondant aux dimensions de la matrice.
    - title (str): Titre du graphique (par défaut 'Matrice de confusion').
    - cmap (matplotlib.colors.Colormap): Carte des couleurs à utiliser pour le graphique (par défaut plt.cm.Blues).
	"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('Vérité terrain')
    plt.xlabel('Prédictions')
    plt.tight_layout()
    plt.show()

Pour pouvoir sauvegarder sur votre répertoire Google Drive, il est nécessaire de fournir une autorisation. Pour cela il suffit d'éxecuter la ligne suivante et de saisir le code donné par Google.

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Corriger éventuellement la ligne ci-dessous pour mettre le chemin vers un répertoire spécifique dans votre répertoire Google Drive :

In [ ]:
import sys
my_local_drive='/content/gdrive/MyDrive/Colab Notebooks/ML_FDS'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

####Les jeux de données


Récupération des jeux de données :      

In [ ]:
!wget https://www.lirmm.fr/~poncelet/Ressources/Tiger-Fox-Elephant.zip

In [ ]:
import zipfile
print("Répertoire actuel :", os.getcwd())
with zipfile.ZipFile("Tiger-Fox-Elephant.zip","r") as zip_ref:
    zip_ref.extractall("/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/Data_Project")


Il y a trois jeux de données différents : des tigres, des éléphants et des renards. Pour chacun d'entre eux il y a un ensemble d'images positive et un ensemble d'images négatives. Par exemple dans le répertoire *tiger* il n'y a que des images de tigre et dans le répertoire *Tiger_negative_class* il n'y a que des images d'animaux qui ne correspondent pas à des tigres.   

Le code ci-dessous permet de visualiser quelques images contenues dans le répertoire *tiger*.

In [ ]:
mypath='Data_Project/Tiger-Fox-Elephant/tiger'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
  images[n] = cv2.imread( join(mypath,onlyfiles[n]) )


COLUMNS = 25 # Nombre d'images à afficher

plt.figure(figsize=(15,15))
for i in range(COLUMNS):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # cv2 lit met les images en BGR et matplotlib lit du RGB
    # il faut donc convertir pour afficher les bonnes couleurs
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
    plt.imshow(images[i],cmap=plt.cm.binary)
    plt.xlabel('taille ' + str(images[i].shape))

Nous pouvons constater que les images ne sont pas de la même taille. Il faut donc les convertir. Une manière simple de faire et de faire la conversion lors de la lecture des images : ici nous convertissons toutes les images en 124x124.

In [ ]:
IMG_SIZE=128
mypath='Data_Project/Tiger-Fox-Elephant/tiger'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
  images[n] = cv2.imread( join(mypath,onlyfiles[n]) )
  images[n]  = cv2.resize(images[n], (IMG_SIZE, IMG_SIZE))

plt.figure(figsize=(15,15))
for i in range(COLUMNS):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # cv2 lit met les images en BGR et matplotlib lit du RGB
    # il faut donc convertir pour afficher les bonnes couleurs
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
    plt.imshow(images[i],cmap=plt.cm.binary)
    plt.xlabel('taille ' + str(images[i].shape))

**Créer le jeu de données**   

Actuellement pour chaque animal nous avons un répertoire qui contient des images positives et un répertoire qui contient des images négatives. Pour pouvoir créer un jeu de données nous devons obtenir X et y. Les fonctions ci-dessous permettent de générer, à partir des répertoires, un jeu de données aléatoire pour X et y.

In [ ]:
def create_training_data(path_data, list_classes):
  training_data=[]
  for classes in list_classes:
      path=os.path.join(path_data, classes)
      class_num=list_classes.index(classes)
      for img in os.listdir(path):
        try:
          img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_UNCHANGED)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
          training_data.append([new_array, class_num])
        except Exception as e:
          pass
  return training_data

def create_X_y (path_data, list_classes):
      # récupération des données
      training_data=create_training_data(path_data, list_classes)
      # tri des données
      #random.shuffle(training_data)
      # création de X et y
      X=[]
      y=[]
      for features, label in training_data:
        X.append(features)
        y.append(label)
      X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
      y=np.array(y)
      return X,y

def plot_examples(X,y):
  plt.figure(figsize=(15,15))
  for i in range(COLUMNS):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # cv2 lit met les images en BGR et matplotlib lit du RGB
    X[i] = cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB)
    plt.imshow(X[i]/255.,cmap=plt.cm.binary)
    plt.xlabel('classe ' + str(y[i]))

Définition de constante globale      


In [ ]:
# constantes globales

IMG_SIZE=128
COLUMNS = 25 # Nombre d'images à afficher

Pour les tigres :

In [ ]:
my_path="Data_Project/Tiger-Fox-Elephant/"
my_classes=['tiger','Tiger_negative_class']
X,y=create_X_y (my_path,my_classes)
print ("Nombre de données : ",X.shape[0])
print ("Taille d'une image pour connaître l'input du réseau", X[0].shape)
#print ("Distribution des labels dans le jeu d'apprentissage")
#sns.countplot(np.array(y))
#plt.title("Nombre d'éléments par classe")
# affichage
plot_examples(X,y)

# Surtout ne pas oublier de normaliser les données avec :
X=X.astype('float')
X=X/255.0

Pour les éléphants :     

In [ ]:
my_path="Data_Project/Tiger-Fox-Elephant/"
my_classes=['elephant','Elephant_negative_class']
X,y=create_X_y (my_path,my_classes)
print ("Nombre de données : ",X.shape[0])
print ("Taille d'une image pour connaître l'input du réseau", X[0].shape)
#print ("Distribution des labels dans le jeu d'apprentissage")
#sns.countplot(np.array(y))
#plt.title("Nombre d'éléments par classe")
# affichage
plot_examples(X,y)

# Surtout ne pas oublier de normaliser les données avec :
X=X.astype('float')
X=X/255.0

Pour les renards :     


In [ ]:
my_path="Data_Project/Tiger-Fox-Elephant/"
my_classes=['fox','Fox_negative_class']
X,y=create_X_y (my_path,my_classes)
print ("Nombre de données : ",X.shape[0])
print ("Taille d'une image pour connaître l'input du réseau", X[0].shape)
#print ("Distribution des labels dans le jeu d'apprentissage")
#sns.countplot(np.array(y))
#plt.title("Nombre d'éléments par classe")
# affichage
plot_examples(X,y)

# Surtout ne pas oublier de normaliser les données avec :
X=X.astype('float')
X=X/255.0

In [ ]:
# constantes globales

IMG_SIZE=128
COLUMNS = 25 # Nombre d'images à afficher

def get_optimizer_info(model):
    optimizer_config = model.optimizer.get_config()
    optimizer_name = model.optimizer.__class__.__name__  #Récupère le nom de l'optimiseur
    optimizer_params = f"lr={optimizer_config['learning_rate']:.5f}, momentum={optimizer_config.get('momentum', 'N/A')}"

    return optimizer_name, optimizer_params
def save_to_csv(data, model, hyperparams=None, filename='/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/resultats_evaluation.csv'):
    optimizer_name, optimizer_params = get_optimizer_info(model)
    file_exists = os.path.exists(filename)
    print(">>> Chemin absolu utilisé :", os.path.abspath(filename))
    print(">>> Existe déjà ?", os.path.exists(filename))
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        if not file_exists:
            header = [
                'Folds', 'Epochs', 'Batch_Size',
                'Val_Loss', 'Val_Accuracy', 'Std_Accuracy',
                'Model_Type', 'Nom_Modele', 'Dropout_Rate',
                'Optimizer_Name', 'Optimizer_Params','Hyperparam'
            ]
            writer.writerow(header)

        if hyperparams:
            hyperparams_str = ', '.join(f"{k}={v}" for k, v in hyperparams.items())
        else:
            hyperparams_str = "NULL"

        row = data + [optimizer_name, optimizer_params, hyperparams_str]
        writer.writerow(row)
        file.flush()
        os.fsync(file.fileno())
        print("Fichier CSV attendu :", os.path.abspath(filename))
        print("Existe déjà ?", os.path.exists(filename))
        print("Ligne à écrire :", row)
        print("Données enregistrées")

def build_dataframe_from_dir(data_dir, class_list):
    data = []
    for class_name in class_list:
        folder_path = os.path.join(data_dir, class_name)
        for fname in os.listdir(folder_path):
            if fname.lower().endswith('.jpg'):
                data.append({
                    'filepath': os.path.join(folder_path, fname),
                    'class': class_name
                })
    return pd.DataFrame(data)

def evaluate_model(model_builder, dataX, dataY, folds=5, epochs=10, batch_size=16):
    scores, histories = [], []
    kfold = KFold(n_splits=folds, shuffle=True, random_state=42)

    #K-fold
    for train_ix, test_ix in kfold.split(dataX):
        X_train, y_train = dataX[train_ix], dataY[train_ix]
        X_test, y_test = dataX[test_ix], dataY[test_ix]
        #Model vierge
        model = model_builder()
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                            validation_data=(X_test, y_test), verbose=1)
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        print(f'Précision : {acc * 100:.3f}%')

        scores.append(acc)
        histories.append(history)

    return scores, histories

def evaluate_model_from_dataframe(model_builder, df, folds=3, epochs=30, batch_size=16, target_size=(128, 128),class_list=None):
    scores, histories = [], []
    kfold = KFold(n_splits=folds, shuffle=True, random_state=42)
    print("appel nouveau evluate_model")
    for train_idx, test_idx in kfold.split(df):
        train_df = df.iloc[train_idx]
        val_df = df.iloc[test_idx]

        train_datagen = ImageDataGenerator(
          rescale=1./255,
          horizontal_flip=True,
          rotation_range=8,
          width_shift_range=0.15,
          height_shift_range=0.15,
          zoom_range=0.15,
        )

        val_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            x_col='filepath',
            y_col='class',
            classes=class_list, #super important pr avoir nos bonnes classes, sinon ça prend par ordre alpahébtique et on a une inversion
            target_size=target_size,
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True
        )
        print("Mapping des classes Keras :", train_generator.class_indices)
        val_generator = val_datagen.flow_from_dataframe(
            dataframe=val_df,
            x_col='filepath',
            y_col='class',
            classes=class_list, #idem
            target_size=target_size,
            class_mode='binary',
            batch_size=batch_size,
            shuffle=False
        )
        print("Mapping des classes Keras :", val_generator.class_indices)
        #Model vierge
        model = model_builder()

        history = model.fit(
            train_generator,
            validation_data=val_generator,
            steps_per_epoch=np.ceil(train_generator.samples / batch_size).astype(int),
            validation_steps=np.ceil(val_generator.samples / batch_size).astype(int),
            epochs=epochs,
            verbose=1
        )

        loss, acc = model.evaluate(val_generator, verbose=0)
        print(f'Précision : {acc * 100:.3f}%')

        scores.append(acc)
        histories.append(history)

    return scores, histories

def run_evaluation(folds, epochs,model_builder,X_animal=None,y_animal=None,best_params=None,model_path=None,save_final_model=True,batch_size=16,IDG=False,target_size=(128,128),df=None,class_list=None):
    model = model_builder()
    print(model.summary())

    if IDG:
      print("utilise IDG")
      scores, histories = evaluate_model_from_dataframe(
          model_builder=model_builder,
          df=df,
          folds=folds,
          epochs=epochs,
          batch_size=batch_size,
          target_size=target_size,
          class_list=class_list
      )
    else:
        scores, histories = evaluate_model(model_builder,X_animal, y_animal, folds, epochs)
    plot_curves(histories)

    val_loss = np.mean([h.history['val_loss'][-1] for h in histories])
    # val_acc2 = np.mean([h.history['val_accuracy'][-1] for h in histories]) #Normalement faire ça pr recup la val_acc, mais j'me suis rendu compte de l'erruer
    # print(val_acc2)
    #pour une raison que j'ignore val_acc2 a tous le temps la même valeur que np.mean(scores)
    val_acc = np.mean(scores)  #Moyenne de la précision finale sur les K-folds
    #écart-type de la précision finale sur les K-folds
    std_acc = np.std(scores)
    data = [
        folds, epochs, batch_size,
        val_loss, val_acc, std_acc,
        'Convolutionnel', global_model_name, "NULL"
    ]
    if save_final_model and model_path:
      save_to_csv(data, model, hyperparams=best_params)
      final_model = model_builder()
      if IDG:
        datagen = ImageDataGenerator(rescale=1./255)
        full_generator = datagen.flow_from_dataframe(
            df,
            x_col='filepath',
            y_col='class',
            classes=class_list,
            target_size=target_size,
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True
        )
        final_model.fit(full_generator, epochs=epochs)
      else:
        final_model.fit(X_animal, y_animal, epochs=epochs, batch_size=batch_size, verbose=1)
      final_model.save(model_path)
      print(f"Modèle final sauvegardé sous : {model_path}")

    print(f'Précision finale : moyenne={mean(scores) * 100:.3f}% écart-type={std(scores) * 100:.3f}%, k={len(scores)}')

In [ ]:
global_model_name = input("Veuillez insérer le nom de modèle (sans le .keras): ")
global_model_name += '.keras'
save_dir = "saved_models/"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, global_model_name)
animals = {"0": "tiger","1": "fox","2": "elephant"}
animal_choix = input("Veuillez insérer le numéro de l'animal voulu (0=tiger, 1=fox, 2=elephant) : ")
if animal_choix not in animals:
    raise ValueError("Choix invalide. Veuillez entrer 0, 1 ou 2.")
selected_animal = animals[animal_choix]
class_list = [selected_animal, f"{selected_animal.capitalize()}_negative_class"]
X_animal, y_animal = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X_animal = X_animal.astype('float32') / 255.0 #Ici j'ai mis float32 pour avoir moins d'octet et utiliser moins de ressources (16,32,64 possible)

def build_baseline_model(shapeinput=(128, 128, 3), learning_rate=0.01, momentum=0.9):
    model = Sequential()
    model.add(Input(shape=shapeinput, name="Input_Layer"))

    #Couche convolutionnelle avec pooling (1 couche CNN)
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name="Conv2D_1"))
    model.add(MaxPooling2D(pool_size=(2, 2), name="MaxPooling2D_1"))

    #Flatten pour préparer les données pour la partie dense
    model.add(Flatten(name="Flatten"))

    #Couches denses pour la classification
    model.add(Dense(1, activation='sigmoid', name="Output"))

    opt = SGD(learning_rate=0.001, momentum=momentum)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# k=3 # Nombre de folds
# epochs=100 # Nombre d'epochs
# # run_evaluation(k,epochs,build_baseline_model,
# #                X_animal,y_animal,
# #                model_path=model_path,save_final_model=False, #mettre a True si on veut save le modèle final + dans le CSV, False sinon
# #                );
# run_evaluation(k,epochs,build_baseline_model,
#                model_path=model_path,save_final_model=True,
#                df=build_dataframe_from_dir("Data_Project/Tiger-Fox-Elephant/", class_list),
#                batch_size=16,IDG=True,
#                target_size=(128,128),
#                class_list=class_list
#                )

Pour des raisons liées à l'empreinte carbone et à la consommation énergétique, l'entraînement du modèle a été désactivé : vous pouvez simplement charger le modèle préentraîné.
Les modèles sont disponible sur notre git

https://github.com/NicolasCani/ProjetL3

In [ ]:
class_list = ["fox", f"Fox_negative_class"]

X_test, y_test = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X_test = X_test.astype('float32') / 255.0 #Ici j'ai mis float32 pour avoir moins d'octet et utiliser moins de ressources (16,32,64 possible)

# Chargement du modèle sauvegardé
model = load_model('/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/saved_models/FoxBaseline.keras') # a modifié en fonction de celui que vous souhaitez

# Évaluation du modèle sur le jeu de test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

# Affichage des résultats de test
print(f'Loss sur le jeu de test : {test_loss *100:.2f}')
print(f'Accuracy sur le jeu de test : {test_acc * 100:.2f}%')

# Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# Calcul de la matrice de confusion
conf = confusion_matrix(y_test, np.round(y_pred).astype(int))

# Affichage de la matrice de confusion
plot_confusion_matrix(conf, ["Fox_negative_class", "fox"])


In [ ]:
global_model_name = input("Veuillez insérer le nom de modèle (sans le .keras): ")
global_model_name += '.keras'
save_dir = "saved_models/"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, global_model_name)
animals = {"0": "tiger","1": "fox","2": "elephant"}
animal_choix = input("Veuillez insérer le numéro de l'animal voulu (0=tiger, 1=fox, 2=elephant) : ")
if animal_choix not in animals:
    raise ValueError("Choix invalide. Veuillez entrer 0, 1 ou 2.")
selected_animal = animals[animal_choix]
class_list = [selected_animal, f"{selected_animal.capitalize()}_negative_class"]
X_animal, y_animal = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X_animal = X_animal.astype('float32') / 255.0

def build_model_Q2(shapeinput=(128, 128, 3),
                num_cnn_layers=2,filters_list=[32, 64],kernel_sizes=[(3, 3), (3, 3)],
                num_dense_layers=2, dense_units_list=[64, 32],
                dropout_rate=0.3,
                learning_rate=0.01,
                momentum=0.9,
                   ):
    print("Paramètres (choisis par Optuna):")
    print("learning_rate :", learning_rate)
    print("momentum :", momentum)
    print("dropout_rate :", dropout_rate)
    print("CNN layers :", num_cnn_layers, filters_list, kernel_sizes)
    print("Dense layers :", num_dense_layers, dense_units_list)
    print("--------------")

    model = Sequential()
    model.add(Input(shape=shapeinput, name="Input_Layer"))

    for i in range(num_cnn_layers):
        model.add(Conv2D(filters=filters_list[i], kernel_size=kernel_sizes[i], activation='relu', name=f"Conv2D_{i}"))
        model.add(MaxPooling2D(pool_size=(2, 2), name=f"MaxPooling2D_{i}"))
    model.add(Flatten(name="Flatten"))

    for i in range(num_dense_layers):
        model.add(Dense(dense_units_list[i], activation='relu', name=f"Dense_{i+1}"))
        model.add(Dropout(dropout_rate, name=f"Dropout_{i+1}"))

    model.add(Dense(1, activation='sigmoid', name="Output"))

    opt = SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    learning_rate = 0.001#a augementer si on utilise pas de IDG a réduire sinon
    momentum = trial.suggest_categorical('momentum', [0.6, 0.7, 0.8, 0.9, 0.95])
    dropout_rate = trial.suggest_categorical('dropout_rate', [0.1, 0.2, 0.3, 0.4, 0.5])

    num_cnn_layers = trial.suggest_int('num_cnn_layers', 1, 3)
    filters_list = [trial.suggest_categorical(f'filters_{i}', [16, 32, 64]) for i in range(num_cnn_layers)]
    kernel_sizes = [trial.suggest_categorical(f'kernel_{i}', [(3, 3), (5, 5)]) for i in range(num_cnn_layers)]

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 3)
    dense_units_list = [trial.suggest_categorical(f'units_{i}', [32, 64]) for i in range(num_dense_layers)]

    print("Paramètres choisis par Optuna :")
    print("learning_rate :", learning_rate)
    print("momentum :", momentum)
    print("dropout_rate :", dropout_rate)
    print("CNN layers :", num_cnn_layers, filters_list, kernel_sizes)
    print("Dense layers :", num_dense_layers, dense_units_list)
    print("--------------")

    def model_fn():
        return build_model_Q2(
            learning_rate=learning_rate,
            momentum=momentum,
            dropout_rate=dropout_rate,
            num_cnn_layers=num_cnn_layers,
            filters_list=filters_list,
            kernel_sizes=kernel_sizes,
            num_dense_layers=num_dense_layers,
            dense_units_list=dense_units_list
        )

    scores, _ = evaluate_model(model_fn, X_animal, y_animal, folds=k, epochs=epochs) #MODIFIER ICI SI BESOIN D'UTILISER IDG FAIRE LE BON APPEL

    return np.mean(scores)

# k=3 # Nombre de folds
# epochs=60 # Nombre d'epochs
# # Lancer l’opti
# print("lancement de l'opti")
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10) # Nombre d'essaye a set ici

# best_params = study.best_params
# def best_model_fn():
#     return build_model_Q2(
#         learning_rate=0.001, #a augementer si on utilise pas de IDG a réduire sinon
#         momentum=best_params['momentum'],
#         dropout_rate=best_params['dropout_rate'],
#         num_cnn_layers=best_params['num_cnn_layers'],
#         filters_list=[best_params[f'filters_{i}'] for i in range(best_params['num_cnn_layers'])],
#         kernel_sizes=[best_params[f'kernel_{i}'] for i in range(best_params['num_cnn_layers'])],
#         num_dense_layers=best_params['num_dense_layers'],
#         dense_units_list=[best_params[f'units_{i}'] for i in range(best_params['num_dense_layers'])]
#     )

# print("Meilleur score :", study.best_value)
# print("Meilleurs hyperparamètres :")
# for key, value in best_params.items():
#     print(f" - {key}: {value}")

# # run_evaluation(X_animal,y_animal,k, epochs,
# #                best_model_fn,best_params=study.best_params, #passage de la fonction qui crée le modele avec les meilleurs hyperparam
# #                model_path=model_path,
# #                save_final_model=False, #mettre a True si on veut save le modèle final + dans le CSV, False sinon
# #                );
# run_evaluation(k,epochs,best_model_fn,best_params=best_params,
#                model_path=model_path,save_final_model=True,
#                df=build_dataframe_from_dir("Data_Project/Tiger-Fox-Elephant/", class_list),
#                batch_size=16,IDG=True,
#                target_size=(128,128)
#                )



Pour des raisons liées à l'empreinte carbone et à la consommation énergétique, l'entraînement du modèle a été désactivé : vous pouvez simplement charger le modèle préentraîné. Les modèles sont disponible sur notre git

https://github.com/NicolasCani/ProjetL3


In [ ]:
class_list = ["fox", f"Fox_negative_class"]

X_test, y_test = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X_test = X_test.astype('float32') / 255.0 #Ici j'ai mis float32 pour avoir moins d'octet et utiliser moins de ressources (16,32,64 possible)

# Chargement du modèle sauvegardé
model = load_model('/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/saved_models/FoxEtenduV2.keras') # a modifié en fonction de celui que vous souhaitez

# Évaluation du modèle sur le jeu de test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

# Affichage des résultats de test
print(f'Loss sur le jeu de test : {test_loss *100:.2f}')
print(f'Accuracy sur le jeu de test : {test_acc * 100:.2f}%')

# Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# Calcul de la matrice de confusion
conf = confusion_matrix(y_test, np.round(y_pred).astype(int))

# Affichage de la matrice de confusion
plot_confusion_matrix(conf, ["Fox_negative_class", "fox"])


In [ ]:
#code pour voir les images bien/mal classé
nom_Modele = input("Veuillez écrire le nom du modèle (sans le .keras) : ")
model = load_model(f'saved_models/{nom_Modele}.keras')

animals = {"0": "tiger", "1": "fox", "2": "elephant"}
animal_choix = input("Veuillez insérer le numéro de l'animal voulu (0=tiger, 1=fox, 2=elephant) : ")
if animal_choix not in animals:
    raise ValueError("Choix invalide. Veuillez entrer 0, 1 ou 2.")
selected_animal = animals[animal_choix]

class_list = [selected_animal, f"{selected_animal.capitalize()}_negative_class"]
X, y = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X = X.astype('float32') / 255.0
y = np.asarray(y)


pred_probs = model.predict(X)
prediction = (pred_probs > 0.5).astype(int).flatten()

misclassified = np.where(y != prediction)[0]

print(f"Nb d'images mal classées : {len(misclassified)}")
print("Les images mal classées sont :")

for i in misclassified:
    true_label = class_list[y[i]]
    predicted_label = class_list[prediction[i]]
    print(f"index: {i}, réel: {true_label}, prédiction: {predicted_label}")

print("Images mal classées :")
for i in misclassified:
    img = (X[i] * 255).astype(np.uint8)
    img_rgb = img[:, :, ::-1]
    plt.imshow(img_rgb)
    plt.title(f"index {i} | réel: {class_list[y[i]]} | prédiction: {class_list[prediction[i]]}")
    plt.axis("off")
    plt.show()

```python
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,                # Centrer les données en soustrayant la moyenne
    samplewise_center=False,                 # Centrer chaque échantillon
    featurewise_std_normalization=False,     # Normaliser les données en divisant par l'écart-type
    samplewise_std_normalization=False,      # Normaliser chaque échantillon
    zca_whitening=False,                     # Appliquer le blanchiment ZCA
    zca_epsilon=1e-06,                      # Epsilon pour le blanchiment ZCA
    rotation_range=0,                        # Plage de degrés pour les rotations aléatoires
    width_shift_range=0.0,                  # Fraction de la largeur totale pour les décalages horizontaux aléatoires
    height_shift_range=0.0,                 # Fraction de la hauteur totale pour les décalages verticaux aléatoires
    brightness_range=None,                   # Plage pour les ajustements aléatoires de luminosité
    shear_range=0.0,                         # Angle de cisaillement dans le sens antihoraire
    zoom_range=0.0,                          # Plage pour le zoom aléatoire
    channel_shift_range=0.0,                 # Décalage des canaux de couleur
    fill_mode='nearest',                     # Les points en dehors des limites sont remplis selon le mode choisi
    cval=0.0,                                # Valeur utilisée pour les points en dehors des limites lorsque fill_mode est 'constant'
    horizontal_flip=False,                   # Retourner aléatoirement les entrées horizontalement
    vertical_flip=False,                     # Retourner aléatoirement les entrées verticalement
    rescale=None,                            # Facteur de mise à l'échelle
    preprocessing_function=None,             # Fonction qui sera appliquée à chaque image
    data_format=None,                        # Format des données d'image, soit 'channels_last' soit 'channels_first'
    validation_split=0.0,                   # Fraction des données réservée pour la validation
    dtype=None                               # Type de données
)


In [ ]:
#code juste pour voir les images
animals = {"0": "tiger","1": "fox","2": "elephant"}
animal_choix = input("Veuillez insérer le numéro de l'animal voulu (0=tiger, 1=fox, 2=elephant) : ")
if animal_choix not in animals:
    raise ValueError("Choix invalide. Veuillez entrer 0, 1 ou 2.")
selected_animal = animals[animal_choix]
class_list = [selected_animal, f"{selected_animal.capitalize()}_negative_class"]

X_animal, y_animal = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)

datagen = ImageDataGenerator(
      horizontal_flip=True,
      rotation_range=3,
      width_shift_range=0.02,
      height_shift_range=0.02,
      zoom_range=0.15,
)

nb_images = 10

nb_genere = 0
batch_size = 5

for X_batch, y_batch in datagen.flow(X_animal, y_animal, batch_size=batch_size, shuffle=True):
    for i in range(len(X_batch)):
        img = X_batch[i]
        label = y_batch[i]

        class_name = class_list[label] # 0 dans le cas où classe 0 (donc tigre) 1 si non tigre

        img_rgb = img.astype(np.uint8)
        img_rgb = img_rgb[:, :, ::-1]  # Sinon les tigres sont bleues lol
        image_pil = Image.fromarray(img_rgb)

        # Sauvegarde
        plt.imshow(img_rgb)
        plt.title(f"Image Générée {nb_genere} - Classe: {class_name}")
        plt.axis("off")
        plt.show()

        nb_genere += 1
        if nb_genere >= nb_images:
            break
    if nb_genere >= nb_images:
        break

In [ ]:
global_model_name = input("Veuillez insérer le nom de modèle (sans le .keras): ")
global_model_name += '.keras'
save_dir = "saved_models/"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, global_model_name)
animals = {"0": "tiger", "1": "fox", "2": "elephant"}
animal_choix = input("Veuillez insérer le numéro de l'animal voulu (0=tiger, 1=fox, 2=elephant) : ")
if animal_choix not in animals:
    raise ValueError("Choix invalide. Veuillez entrer 0, 1 ou 2.")

selected_animal = animals[animal_choix]
class_list = [selected_animal, f"{selected_animal.capitalize()}_negative_class"]
X_animal, y_animal = create_X_y("Data_Project/Tiger-Fox-Elephant/", class_list)
X_animal = X_animal.astype('float32') / 255.0 #Ici j'ai mis float32 pour avoir moins d'octet et utiliser moins de ressources (16,32,64 possible)

def modelresnet():
  input_shape_animal = (128, 128, 3)
  input_shape_resnet = (224, 224, 3)
  model = Sequential()

  model.add(Input(shape=input_shape_animal, name="Input_Layer"))

  # Ajouter une couche Lambda pour redimensionner les images
  model.add(Lambda(lambda image: tf.image.resize(image, input_shape_resnet[:2]), name="Lambda_Layer"))  # Resize to (224, 224)
  # Ajouter ResNet50 sans la couche de classification
  model.add(DenseNet121(include_top=False, pooling='avg', weights='imagenet',classes=2,name="DenseNet121_Layer"))
  model.get_layer("DenseNet121_Layer").trainable = False
  # Ajout de la partie classification
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid', name="Output"))
  # Compiler le modèle
  opt = SGD(learning_rate=0.01)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

  # Afficher le résumé du modèle
  return model

# k=3 # Nombre de folds
# epochs=60 # Nombre d'epochs
# # run_evaluation(k, epochs,modelresnet,
# #                X_animal,y_animal,
# #                model_path=model_path,save_final_model=False, #mettre a True si on veut save le modèle final + dans le CSV, False sinon
# #                );
# # model=modelresnet()
# # model.get_layer("ResNet_Layer").trainable = False
# # print(model.summary())
# # history = model.fit(X_animal, y_animal, epochs=epochs, batch_size=64,
# #                             validation_data=(X_animal, y_animal), verbose=1)
# # plot_curves([history])
# run_evaluation(k,epochs,modelresnet,
#                model_path=model_path,save_final_model=True,
#                df=build_dataframe_from_dir("Data_Project/Tiger-Fox-Elephant/", class_list),
#                batch_size=16,IDG=True,
#                target_size=(224,224) #important ça !
#                )

PARTIE AUTOENCODEUR

In [ ]:
# Importation des différentes librairies utiles pour le notebook

# Sickit-learn met régulièrement à jour des versions et
# indique des futurs warnings. Ces deux lignes permettent de ne pas les afficher.
import warnings  # Permet de gérer les avertissements
warnings.filterwarnings("ignore", category=FutureWarning)

# Librairies générales
import pickle  # Pour la sérialisation et la désérialisation d'objets Python
import pandas as pd  # Manipulation de données en format tabulaire
from scipy.stats import randint  # Pour les distributions aléatoires
import numpy as np  # Calcul numérique
from numpy import mean, std  # Statistiques de base
import string  # Manipulation des chaînes de caractères
import os  # Interaction avec le système d'exploitation
from os import listdir  # Liste les fichiers dans un répertoire
from os.path import isfile, join  # Fonctions pour gérer les chemins de fichiers
import time  # Gestion du temps
import base64  # Encodage et décodage en base64
import re  # Expressions régulières
import sys  # Manipulation du système
import copy  # Copie d'objets Python
import random  # Génération de nombres aléatoires
import datetime  # Manipulation des dates et heures
import glob  # Recherche de fichiers avec des motifs
from glob import glob  # Pour la recherche de fichiers avec des motifs
import cv2  # Librairie pour la vision par ordinateur
import zipfile  # Manipulation des archives ZIP

# Librairie d'affichage
import matplotlib.pyplot as plt  # Visualisation de données
from PIL import Image, ImageDraw  # Manipulation d'images avec Pillow

# Scikit-learn pour l'apprentissage automatique
from sklearn.metrics import confusion_matrix, accuracy_score  # Métriques pour l'évaluation des modèles
from sklearn.model_selection import KFold, cross_val_score, train_test_split  # Séparation des données et validation croisée
from sklearn.pipeline import Pipeline  # Création de pipelines pour les modèles
from sklearn.decomposition import PCA  # Analyse en composantes principales (ACP)

# TensorFlow et Keras pour le deep learning
import tensorflow as tf  # Framework principal pour l'apprentissage profond
import keras  # API de haut niveau pour TensorFlow
from keras import layers  # Modules de construction de modèles Keras
from keras import models  # Définition et manipulation des modèles
from keras import optimizers  # Optimisateurs pour les modèles Keras
#from keras.preprocessing.image import ImageDataGenerator  # Déprécié, remplacé par TensorFlow
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Générateur d'images pour l'augmentation des données

# Utilitaires Keras pour le traitement d'images
from keras.utils import img_to_array, load_img  # Conversion d'images en tableaux
from keras.callbacks import ModelCheckpoint, EarlyStopping  # Pour la gestion des checkpoints et de l'arrêt précoce
from keras import backend as K  # Backend Keras pour des opérations de bas niveau
from keras.models import Sequential, Model, load_model  # Définition et chargement des modèles Keras
from keras.layers import LeakyReLU  # Activation LeakyReLU
from keras.preprocessing import image  # Prétraitement des images

# Couches Keras pour les modèles convolutifs et autres
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout  # Couches pour CNN et autoencoders
from keras.layers import InputLayer, MaxPooling2D, UpSampling2D  # Couches supplémentaires pour les CNN
from keras.optimizers import Adam  # Optimiseur Adam
from keras.utils import plot_model  # Visualisation du modèle
import tensorflow as tf
from PIL import Image, ImageDraw
import numpy as np
import random
from glob import glob


In [ ]:
# Définition des paramètres principaux
DATA_FOLDER = "Data_Project/Tiger-Fox-Elephant"
WEIGHTS_FOLDER = './myweights/AE'
WEIGHTS_FOLDER2 = './myweights/AENoise'
WEIGHTS_FOLDER3 = './myweights/AENB'
WEIGHTS_FOLDER4 = './myweights/AEElephant'
WEIGHTS_FOLDER5 = './myweights/AETiger'


# Calcul du nombre d'images dans le dossier spécifié
filenames = np.array(glob(os.path.join(DATA_FOLDER+'/fox', '*.jpg')) + glob(os.path.join(DATA_FOLDER+'/fox', '*.jpeg')))
NUM_IMAGES = len(filenames)
print(NUM_IMAGES)
print("Total number of images : " + str(NUM_IMAGES))

# Création du répertoire pour sauvegarder les poids, s'il n'existe pas
os.makedirs(WEIGHTS_FOLDER, exist_ok=True)
os.makedirs(WEIGHTS_FOLDER2, exist_ok=True)
os.makedirs(WEIGHTS_FOLDER3, exist_ok=True)
os.makedirs(WEIGHTS_FOLDER4, exist_ok=True)
os.makedirs(WEIGHTS_FOLDER5, exist_ok=True)

def add_mask_on_image(image):
    """
    Ajoute un masque sous forme de carré noir sur l'image donnée.

    Le masque est un carré de taille fixe, positionné aléatoirement sur l'image. Ce carré
    cache une partie de l'image originale.

    Paramètres:
    -----------
    image : PIL.Image
        L'image sur laquelle le masque sera ajouté. Doit être une image au format RGB.

    Retour:
    -------
    numpy.ndarray
        L'image modifiée avec un masque noir ajouté sous forme de carré.
    """
    size_square = 42  # Taille du carré
    output_img = image
    random_start_point = np.random.randint(0, 100)  # Point de départ aléatoire pour le carré
    img_draw = ImageDraw.Draw(output_img)
    # Dessiner le carré noir
    img_draw.rectangle([(random_start_point, random_start_point),
                        (random_start_point + size_square, random_start_point + size_square)], fill='black')
    return np.array(output_img)


def add_noise_on_image(image):
    """
    Ajoute du bruit aléatoire à l'image.

    Le bruit est généré selon une distribution normale et est ajouté à l'image.
    Plus le niveau de bruit est élevé, plus l'image est déformée.

    Paramètres:
    -----------
    image : PIL.Image
        L'image à laquelle du bruit sera ajouté. Doit être une image au format RGB.

    Retour:
    -------
    PIL.Image
        L'image bruitée.
    """
    level_of_noise = 4  # Niveau de bruit. Plus il est élevé, plus la déformation est importante
    # Convertir l'image en tableau NumPy
    image_array = np.array(image)
    # Générer du bruit aléatoire avec la même forme que l'image
    noise = np.random.normal(loc=0, scale=25 * level_of_noise, size=image_array.shape)
    # Ajouter le bruit à l'image
    noisy_image_array = image_array + noise
    # Limiter les valeurs pour qu'elles soient dans la plage valide (0-255)
    noisy_image_array = np.clip(noisy_image_array, 0, 255)
    # Convertir le tableau NumPy en image
    noisy_image = Image.fromarray(noisy_image_array.astype(np.uint8))
    return noisy_image


def black_and_white_on_image(image):
    """
    Convertit l'image en niveaux de gris et redimensionne à 128x128.

    Cette fonction prend une image en couleur, la convertit en niveaux de gris et
    la redimensionne à une taille fixe de 128x128.

    Paramètres:
    -----------
    image : PIL.Image
        L'image à convertir en noir et blanc. Doit être une image au format RGB.

    Retour:
    -------
    numpy.ndarray
        L'image en niveaux de gris redimensionnée à 128x128 et convertie en tableau NumPy.
    """
    # Convertir l'image en niveaux de gris
    grayscale_image = image.convert("L")
    # Redimensionner l'image en niveaux de gris à la taille souhaitée (128x128)
    grayscale_image = grayscale_image.resize((128, 128))
    # Convertir l'image en niveaux de gris en tableau NumPy pour l'affichage avec matplotlib
    image_array = np.array(grayscale_image)
    # Empiler le tableau en niveaux de gris pour créer une image en niveaux de gris à 3 canaux
    image_array = np.stack((image_array,) * 3, axis=-1)
    return image_array


def my_image_generator(folder, classes, target_size, batch_size, typeimage='mask', shuffle=True):
    """
    Génère un batch d'images corrompues et originales pour l'entraînement d'un modèle.

    En fonction du type d'image spécifié (mask, noise, blackwhite), la fonction génère des
    images corrompues en ajoutant des masques, du bruit ou en convertissant les images en noir et blanc.
    Les images originales sont également renvoyées pour l'entraînement supervisé.

    Paramètres:
    -----------
    folder : str
        Le répertoire où se trouvent les sous-dossiers d'images de chaque classe.
    classes : list
        Liste des classes d'images (chaque classe étant un sous-dossier dans `folder`).
    target_size : tuple
        Taille cible des images redimensionnées (hauteur, largeur).
    batch_size : int
        Taille du batch (nombre d'images à traiter par itération).
    typeimage : str, optionnel
        Type de transformation à appliquer sur les images corrompues. Les valeurs possibles sont :
        - 'mask' : ajouter un masque sous forme de carré.
        - 'noise' : ajouter du bruit aléatoire.
        - 'blackwhite' : convertir l'image en noir et blanc.
    shuffle : bool, optionnel
        Si True, les fichiers sont mélangés avant d'être utilisés.

    Retour:
    -------
    tuple
        Un tuple contenant deux éléments :
        - Un tableau numpy des images corrompues.
        - Un tableau numpy des images originales.
    """
    # Créer une liste de chemins de fichiers
    files = []
    for c in classes:
        class_folder = folder + '/' + c
        file_pattern = class_folder + '/*.jpg'  # Mettre à jour l'extension des fichiers pour correspondre à vos images
        files.extend(glob(file_pattern))

    # Mélanger la liste des chemins de fichiers si shuffle est True
    if shuffle:
        random.shuffle(files)

    # Initialiser le compteur d'itérations
    iter = 0

    while True:
        # Vérifier si l'index des fichiers est invalide, si oui réinitialiser
        if (iter * batch_size + batch_size) > len(files):
            iter = 0
        end_iter = iter * batch_size + batch_size
        # Sélectionner les fichiers pour le batch
        paths = files[iter * batch_size:end_iter]
        # Lire chaque image et effectuer le prétraitement pour le batch d'images
        corrupted_images_batch = []
        original_images_batch = []
        for path in paths:
            img = Image.open(path)
            # Ne considérer que les images avec 3 canaux (RGB)
            if img.mode == 'RGB':
                # Redimensionner l'image à la taille cible
                img = img.resize(target_size, resample=Image.BILINEAR)

                original_images_batch.append(np.array(img) / 255)

                # Vérifier la taille de l'image
                if img.size[0] == target_size[0] and img.size[1] == target_size[1]:
                    if typeimage == 'mask':
                        changed_img = add_mask_on_image(img)
                    elif typeimage == 'noise':
                        changed_img = add_noise_on_image(img)
                    elif typeimage == 'blackwhite':
                        changed_img = black_and_white_on_image(img)
                else:
                    print("L'image {} n'a pas les dimensions appropriées.".format(path))

                preprocess_img = np.array(changed_img)
                corrupted_images_batch.append(preprocess_img / 255)
            else:
                continue
        # Retourner un tuple de (batch_images_corrupted, batch_images_originales) pour alimenter le réseau
        corrupted_images_batch = np.array(corrupted_images_batch)
        original_images_batch = np.array(original_images_batch)
        # Passer au batch suivant
        iter = iter + 1

        yield (corrupted_images_batch, original_images_batch)
def display_images_from_batch(batch, typeimage):
    """
    Affiche un batch d'images corrompues et originales sous forme de sous-graphes.

    Cette fonction affiche les 8 premières images du batch donné, en comparant les images originales
    (non modifiées) avec les images corrompues (modifiées). Elle prend en charge les images en noir et blanc
    en ajustant le colormap ('gray').

    Paramètres:
    -----------
    batch : tuple
        Un tuple contenant deux éléments :
        - Le premier élément est une liste ou un tableau d'images corrompues (par exemple, avec des trous, du bruit, etc.).
        - Le second élément est une liste ou un tableau d'images originales, correspondant aux images non modifiées.
        Les deux éléments doivent être des tableaux de forme (batch_size, hauteur, largeur, canaux).

    typeimage : str
        Un indicateur de type d'image :
        - 'blackwhite' : pour afficher les images corrompues en niveaux de gris.
        - Toute autre valeur : pour afficher les images corrompues avec les couleurs d'origine.

    Retour:
    -------
    None
        La fonction affiche les images, mais ne retourne rien.

    Exemples:
    ---------
    1. Pour afficher les premières 8 images d'un batch avec des images corrompues en noir et blanc :
       display_images_from_batch(batch, typeimage='blackwhite')

    2. Pour afficher les premières 8 images d'un batch avec des images corrompues en couleur :
       display_images_from_batch(batch, typeimage='color')
    """

    # Extraire les 8 premières images corrompues du batch
    corrupted_images = batch[0][:8]
    # Extraire les 8 premières images originales du batch
    original_images = batch[1][:8]

    # Créer une figure avec 8 lignes et 2 colonnes pour afficher les images
    fig, axs = plt.subplots(8, 2, figsize=(8, 16))  # Ajustez la taille de la figure
    fig.tight_layout(pad=2.0)  # Ajuster l'espacement entre les sous-graphes

    for i in range(8):
        # Afficher l'image originale dans la première colonne
        axs[i, 0].imshow(original_images[i].squeeze())
        axs[i, 0].set_title('Image Originale')  # Titre de la première colonne
        axs[i, 0].axis('off')  # Masquer les axes

        # Si l'image est en noir et blanc, utiliser le cmap 'gray' pour l'affichage
        if typeimage == 'blackwhite':
            axs[i, 1].imshow(corrupted_images[i].squeeze(), cmap='gray')
        else:
            # Afficher l'image corrompue dans la deuxième colonne
            axs[i, 1].imshow(corrupted_images[i].squeeze())
        axs[i, 1].set_title('Image Corrompue')  # Titre de la deuxième colonne
        axs[i, 1].axis('off')  # Masquer les axes

    plt.show()  # Afficher la figure
def show_images(autoencoder, images=None):
    """
    Fonction pour afficher les images originales et leurs reconstructions générées par l'autoencodeur.

    Si aucun lot d'images n'est fourni, la fonction récupère un lot d'images depuis le générateur de données.

    Arguments :
    - autoencoder : L'autoencodeur entraîné, utilisé pour générer des reconstructions des images.
    - images (optionnel) : Un tableau d'images à afficher. Si non spécifié, un lot d'images est récupéré du générateur de données.

    Sortie :
    - Affichage des images originales et leurs reconstructions générées par l'autoencodeur.
    """

    if images is None:
        example_batch = next(data_flow)
        example_batch = example_batch[0]
        images = example_batch[:10]

    n_to_show = images.shape[0]

    # Génération des reconstructions des images
    reconst_images = autoencoder.predict(images)

    # Création de la figure pour l'affichage
    fig = plt.figure(figsize=(15, 3))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    # Affichage des images originales
    for i in range(n_to_show):
        img = images[i].squeeze()
        sub = fig.add_subplot(2, n_to_show, i+1)
        sub.axis('off')
        sub.imshow(img)

    # Affichage des images reconstruites
    for i in range(n_to_show):
        img = reconst_images[i].squeeze()
        sub = fig.add_subplot(2, n_to_show, i+n_to_show+1)
        sub.axis('off')
        sub.imshow(img)


In [ ]:
INPUT_DIM = (128, 128, 3)   # Dimensions des images (hauteur, largeur, canaux)
BATCH_SIZE = 32            # Taille des lots
LATENT_DIM = 200            # Dimension du vecteur latent pour l'autoencodeur
# Z_DIM définit la dimension de l'espace latent
Z_DIM = 200  # Dimension de l'espace latent

# Génération des données par lot avec normalisation des pixels (valeurs entre 0 et 1)
data_flow = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory=DATA_FOLDER,
    classes=['fox'],
    target_size=INPUT_DIM[:2],  # Redimensionne les images en 128x128
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    class_mode='input'  # Dans un autoencodeur, les entrées et sorties sont identiques
)

# Obtention du premier batch d'images générées
data_batch = next(data_flow)

# Affichage des 30 premières images du batch
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 10, figsize=(12, 6))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(data_batch[0][i])
    ax.axis('off')
plt.show()
print(f"Taille du batch généré : {data_batch[0].shape}")

In [ ]:

# ENCODER
def build_encoder(input_dim, output_dim):
    # En utilisant clear_session, nous supprimons toutes les exécutions précédentes. Cela permet de réinitialiser le modèle.
    tf.keras.backend.clear_session()  # Utilisation de tensorflow.keras.backend pour clear_session

    # Définition de l'entrée du modèle
    encoder_input = Input(shape=input_dim, name='encoder_input')
    x = encoder_input

    # Première convolution
    x = Conv2D(filters=32,
               kernel_size=3,
               strides=2,
               padding='same', name='encoder_conv_1')(x)
    # Application de la fonction d'activation
    x = LeakyReLU()(x)

    # Deuxième convolution
    x = Conv2D(filters=64,
               kernel_size=3,
               strides=2,
               padding='same', name='encoder_conv_2')(x)
    # Application de la fonction d'activation
    x = LeakyReLU()(x)

    # Troisième convolution
    x = Conv2D(filters=64,
               kernel_size=3,
               strides=2,
               padding='same', name='encoder_conv_3')(x)
    # Application de la fonction d'activation
    x = LeakyReLU()(x)

    # Quatrième convolution
    x = Conv2D(filters=64,
               kernel_size=3,
               strides=2,
               padding='same', name='encoder_conv_4')(x)
    # Application de la fonction d'activation
    x = LeakyReLU()(x)

    # Nous devons obtenir la forme de la sortie avant de l'aplatir pour l'envoyer au décodeur
    shape_before_flattening = tf.keras.backend.int_shape(x)[1:]  # Correction ici : tf.keras.backend.int_shape

    # Aplatissement de la dernière couche CNN
    x = Flatten()(x)

    # Définition de la sortie du modèle
    encoder_output = Dense(output_dim, name='encoder_output')(x)

    # Retourne l'entrée, la sortie, la forme avant aplatissement et le modèle complet
    return encoder_input, encoder_output, shape_before_flattening, Model(encoder_input, encoder_output)

    # DECODER

def build_decoder(input_dim, shape_before_flattening):
  # Définir l'entrée du modèle
  decoder_input = Input(shape = (input_dim,) , name = 'decoder_input')

  # Pour obtenir une image miroir exacte de l'encodeur
  x = Dense(np.prod(shape_before_flattening))(decoder_input)  # Convertir le vecteur en une matrice
  x = Reshape(shape_before_flattening)(x)  # Redimensionner en la forme d'avant le Flatten

  # Première couche Conv2DTranspose
  x = Conv2DTranspose(filters = 64,
                  kernel_size = 3,
                  strides = 2,
                  padding = 'same',
                  name = 'decoder_conv_1'
                  )(x)
  # Application de la fonction d'activation
  x = LeakyReLU()(x)

  # Deuxième couche Conv2DTranspose
  x = Conv2DTranspose(filters = 64,
                  kernel_size = 3,
                  strides = 2,
                  padding = 'same',
                  name = 'decoder_conv_2'
                  )(x)
  # Application de la fonction d'activation
  x = LeakyReLU()(x)

  # Troisième couche Conv2DTranspose
  x = Conv2DTranspose(filters = 32,
                  kernel_size = 3,
                  strides = 2,
                  padding = 'same',
                  name = 'decoder_conv_3'
                  )(x)
  # Application de la fonction d'activation
  x = LeakyReLU()(x)

  # Quatrième couche Conv2DTranspose
  x = Conv2DTranspose(filters = 3,
                  kernel_size = 3,
                  strides = 2,
                  padding = 'same',
                  name = 'decoder_conv_4'
                  )(x)
  # Application de la fonction d'activation
  x = Activation('sigmoid')(x)  # Activation finale pour obtenir une sortie entre 0 et 1

  # Sortie du décodeur
  decoder_output = x

  return decoder_input, decoder_output, Model(decoder_input, decoder_output)

In [ ]:

# Création de l'encodeur avec les dimensions d'entrée et de sortie spécifiées
encoder_input, encoder_output, shape_before_flattening, encoder  = build_encoder(input_dim = INPUT_DIM,
                                    output_dim = Z_DIM,
                                    )

encoder.summary()


In [ ]:
# Création du décodeur en utilisant la dimension de l'espace latent et la forme avant le flattening
decoder_input, decoder_output, decoder = build_decoder(input_dim = Z_DIM,
                                        shape_before_flattening = shape_before_flattening
                                        )

decoder.summary()

In [ ]:
# L'entrée du modèle sera l'image envoyée à l'encodeur.
autoencoder_input = encoder_input

# La sortie sera celle du décodeur. Le terme - decoder(encoder_output)
# permet de combiner les deux modèles en envoyant la sortie de l'encodeur comme entrée du décodeur.
autoencoder_output = decoder(encoder_output)

# L'entrée du modèle combiné sera donc l'entrée de l'encodeur.
# La sortie du modèle combiné sera la sortie du décodeur.
autoencoder = Model(autoencoder_input, autoencoder_output)


autoencoder.summary()

In [ ]:
# parameters of the model
LEARNING_RATE=.0005
steps_per_epoch=int(NUM_IMAGES / BATCH_SIZE)
epochs=0
initial_epoch=0

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AE

epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) (0 si aucun fichier existant) : "))
epochs = int(input("Entrez le numéro d'epochs à faire en plus (ex: 25) : "))
filename = f"AE.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER, filename)

# === Construction du modèle (toujours reconstruit, puis poids chargés si besoin) ===

# Recréation propre du modèle
encoder_input, encoder_output, shape_before_flattening, encoder = build_encoder(INPUT_DIM, Z_DIM)
decoder_input, decoder_output, decoder = build_decoder(Z_DIM, shape_before_flattening)

autoencoder_input = encoder_input
autoencoder_output = decoder(encoder_output)
autoencoder = Model(autoencoder_input, autoencoder_output)

if epoch_number == 0:
    print("Nouveau modèle créé (pas de poids chargés)")
else:
    if os.path.exists(filepath):
        autoencoder.load_weights(filepath)
        print(f"Poids chargés depuis : {filepath}")
    else:
        print(f"Fichier introuvable : {filepath}")
        sys.exit(1)
initial_epoch = epoch_number
# Compilation de l'autoencodeur avec l'optimiseur Adam et un taux d'apprentissage spécifié,
# ainsi que l'utilisation de la perte 'mean_squared_error' et de l'exactitude comme métrique
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error' , metrics=['accuracy'])

# Création des données de validation pour l'affichage

val_data_flow=tion_data = next(data_flow)

# Définition d'un point de contrôle pour sauvegarder les poids du modèle pendant l'entraînement
# Le modèle sauvegardera uniquement les poids dans un fichier .h5
checkpoint_autoencoder = ModelCheckpoint(os.path.join(WEIGHTS_FOLDER, 'AE.{epoch:02d}.weights.h5'), save_weights_only = True, verbose=1, save_freq=steps_per_epoch * 25)

# Entraînement de l'autoencodeur avec les données d'entraînement,
# en utilisant les callbacks pour sauvegarder les poids du modèle à chaque époque
history=autoencoder.fit(data_flow,
                     shuffle=True,
                     epochs = initial_epoch+epochs,
                     initial_epoch = initial_epoch,
                     steps_per_epoch=steps_per_epoch,
                     validation_data=val_data_flow,
                     callbacks=[checkpoint_autoencoder])


In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AE
epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) : "))

filename = f"AE.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER, filename)

if os.path.exists(filepath):
    autoencoder.load_weights(filepath)
    print(f"Poids chargés depuis : {filepath}")
else:
    print(f"Fichier introuvable : {filepath}")
    sys.exit(1)
# Récupère le premier lot d'images du générateur de données
example_batch = next(data_flow)

# Sélectionne uniquement les images du batch (le premier élément contient les images)
example_batch = example_batch[0]

# Prend les 10 premières images du lot pour les afficher
example_images = example_batch[:10]

# Affiche les images originales et reconstruites par l'autoencodeur
show_images(autoencoder, example_images)

In [ ]:
from sklearn.manifold import TSNE
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from tensorflow.keras.utils import array_to_img
def visualize_latent_space_fox(encoder_model, data_flow, num_images=100, image_size=(64, 64), zoom=0.8):
    """
    Affiche les renards dans l’espace latent à partir de l’encodeur.

    Paramètres :
    ------------
    encoder_model : keras.Model
        Le modèle encodeur (extrait du modèle autoencodeur entraîné).
    data_flow : ImageDataGenerator.flow_from_directory
        Le générateur de données (ici contenant uniquement des images de renards).
    num_images : int
        Nombre d’images à afficher.
    image_size : tuple
        Taille des vignettes.
    zoom : float
        Zoom appliqué à chaque vignette.
    """
    # Charger toutes les images nécessaires (en 1 ou plusieurs batches)
    X_images = []
    loaded = 0
    while loaded < num_images:
        batch = next(data_flow)[0]
        remaining = num_images - loaded
        if batch.shape[0] > remaining:
            X_images.extend(batch[:remaining])
            break
        else:
            X_images.extend(batch)
            loaded += batch.shape[0]
    X_images = np.array(X_images)

    # Encodage avec le modèle
    latent_vectors = encoder_model.predict(X_images, verbose=0)
    if latent_vectors.ndim > 2:
        latent_vectors = latent_vectors.reshape((latent_vectors.shape[0], -1))

    # Réduction de dimension via t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    latent_2d = tsne.fit_transform(latent_vectors)

    # Affichage
    plt.figure(figsize=(15, 12))
    ax = plt.gca()
    for i, (x, y) in enumerate(latent_2d):
        thumbnail = array_to_img(X_images[i])
        thumbnail = thumbnail.resize(image_size)
        imagebox = OffsetImage(thumbnail, zoom=zoom)
        ab = AnnotationBbox(imagebox, (x, y), frameon=False)
        ax.add_artist(ab)

    ax.set_xlim(latent_2d[:, 0].min() - 5, latent_2d[:, 0].max() + 5)
    ax.set_ylim(latent_2d[:, 1].min() - 5, latent_2d[:, 1].max() + 5)
    plt.title("Répartition des images de renards dans l’espace latent")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    ax.grid(False)
    plt.show()

visualize_latent_space_fox(encoder_model=encoder, data_flow=data_flow, num_images=100)


Noising

In [ ]:
BATCH_SIZE = 12
INPUT_DIM = (128,128,3)
classes=['fox']
target_size=INPUT_DIM[:2]
typeimage='noise'
# Create the generator
mydataflow=my_image_generator(folder=DATA_FOLDER, classes=classes, target_size=target_size,batch_size=BATCH_SIZE, typeimage=typeimage)

# Get a batch
batch = next(mydataflow)
display_images_from_batch(batch,typeimage )

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AENoise

epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) (0 si aucun fichier existant) : "))
epochs = int(input("Entrez le numéro d'epochs à faire en plus (ex: 25) : "))
filename = f"AENoise.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER2, filename)

# === Construction du modèle (toujours reconstruit, puis poids chargés si besoin) ===

# Recréation propre du modèle
encoder_input, encoder_output, shape_before_flattening, encoder = build_encoder(INPUT_DIM, Z_DIM)
decoder_input, decoder_output, decoder = build_decoder(Z_DIM, shape_before_flattening)

autoencoder_input = encoder_input
autoencoder_output = decoder(encoder_output)
autoencoder = Model(autoencoder_input, autoencoder_output)

if epoch_number == 0:
    print("Nouveau modèle créé (pas de poids chargés)")
else:
    if os.path.exists(filepath):
        autoencoder.load_weights(filepath)
        print(f"Poids chargés depuis : {filepath}")
    else:
        print(f"Fichier introuvable : {filepath}")
        sys.exit(1)
initial_epoch = epoch_number
# Compilation du modèle autoencodeur avec un optimiseur Adam et une fonction de perte 'mean_squared_error'
# L'optimiseur utilise un taux d'apprentissage défini par la variable LEARNING_RATE
# Le modèle sera évalué avec l'accuracy comme métrique
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error', metrics=['accuracy'])

# Création des données de validation pour l'affichage
# Récupération du premier batch du générateur de données
val_data_flow = next(mydataflow)

# Définition d'un callback de sauvegarde des poids du modèle
# Les poids seront sauvegardés sous le nom 'AEweights_corrupted.h5' dans le dossier spécifié par WEIGHTS_FOLDER
# Les poids sont sauvegardés à chaque époque et le mode verbose est activé pour afficher des informations pendant l'entraînement
checkpoint_autoencoder = ModelCheckpoint(os.path.join(WEIGHTS_FOLDER2, 'AENoise.{epoch:02d}.weights.h5'), save_weights_only=True, verbose=1,save_freq=steps_per_epoch * 25)

# Entraînement du modèle autoencodeur

history = autoencoder.fit(mydataflow,
                          shuffle=True,
                          epochs = initial_epoch+epochs,
                          initial_epoch = initial_epoch,
                          steps_per_epoch=steps_per_epoch,
                          validation_data=val_data_flow,
                          callbacks=[checkpoint_autoencoder])

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AENoise
epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) : "))

filename = f"AENoise.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER2, filename)

if os.path.exists(filepath):
    autoencoder.load_weights(filepath)
    print(f"Poids chargés depuis : {filepath}")
else:
    print(f"Fichier introuvable : {filepath}")
    sys.exit(1)

example_batch = next(mydataflow)
example_batch = example_batch[0]
example_images = example_batch[:10]
show_images(autoencoder,example_images)

Noir et blanc

In [ ]:
BATCH_SIZE = 12
INPUT_DIM = (128,128,3)
classes=['fox']
target_size=INPUT_DIM[:2]
typeimage='blackwhite'
# Create the generator
mydataflow=my_image_generator(folder=DATA_FOLDER, classes=classes, target_size=target_size,batch_size=BATCH_SIZE, typeimage=typeimage)

# Get a batch
batch = next(mydataflow)
display_images_from_batch(batch,typeimage)



In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AENB

epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) (0 si aucun fichier existant) : "))
epochs = int(input("Entrez le numéro d'epochs à faire en plus (ex: 25) : "))
filename = f"AENB.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER3, filename)

# === Construction du modèle (toujours reconstruit, puis poids chargés si besoin) ===

# Recréation propre du modèle
encoder_input, encoder_output, shape_before_flattening, encoder = build_encoder(INPUT_DIM, Z_DIM)
decoder_input, decoder_output, decoder = build_decoder(Z_DIM, shape_before_flattening)

autoencoder_input = encoder_input
autoencoder_output = decoder(encoder_output)
autoencoder = Model(autoencoder_input, autoencoder_output)

if epoch_number == 0:
    print("Nouveau modèle créé (pas de poids chargés)")
else:
    if os.path.exists(filepath):
        autoencoder.load_weights(filepath)
        print(f"Poids chargés depuis : {filepath}")
    else:
        print(f"Fichier introuvable : {filepath}")
        sys.exit(1)
initial_epoch = epoch_number

# Compilation du modèle autoencodeur avec un optimiseur Adam et une fonction de perte 'mean_squared_error'
# L'optimiseur utilise un taux d'apprentissage défini par la variable LEARNING_RATE
# Le modèle sera évalué avec l'accuracy comme métrique
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error', metrics=['accuracy'])

# Création des données de validation pour l'affichage
# Récupération du premier batch du générateur de données
val_data_flow = next(mydataflow)

# Définition d'un callback de sauvegarde des poids du modèle
# Les poids seront sauvegardés sous le nom 'AEweights_corrupted.h5' dans le dossier spécifié par WEIGHTS_FOLDER
# Les poids sont sauvegardés à chaque époque et le mode verbose est activé pour afficher des informations pendant l'entraînement
checkpoint_autoencoder = ModelCheckpoint(os.path.join(WEIGHTS_FOLDER3, 'AENB.{epoch:02d}.weights.h5'), save_weights_only=True, verbose=1,save_freq=steps_per_epoch * 25)

# Entraînement du modèle autoencodeur

history = autoencoder.fit(mydataflow,
                          shuffle=True,
                          epochs = initial_epoch+epochs,
                          initial_epoch = initial_epoch,
                          steps_per_epoch=steps_per_epoch,
                          validation_data=val_data_flow,
                          callbacks=[checkpoint_autoencoder])

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AENB
epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) : "))

filename = f"AENB.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER3, filename)

if os.path.exists(filepath):
    autoencoder.load_weights(filepath)
    print(f"Poids chargés depuis : {filepath}")
else:
    print(f"Fichier introuvable : {filepath}")
    sys.exit(1)

example_batch = next(mydataflow)
example_batch = example_batch[0]
example_images = example_batch[:10]
show_images(autoencoder,example_images)

In [ ]:
# Create a separate encoder
input_encoder, output_encoder,  _, encoder_model  = build_encoder(input_dim = INPUT_DIM,
                                    output_dim = Z_DIM,
                                    )
#encoder_input, encoder_output, _, encoder_model = build_encoder(input_dim, output_dim)
# Create a batch as X_test
X_test = next(mydataflow)
# Get the corrupted image
X_test = X_test[0]

# Get the latent space
latent_codes = encoder_model.predict(X_test)

# Use PCA to get the 2 main components
pca = PCA(n_components=2)
latent_codes_2d = pca.fit_transform(latent_codes)

# Plot the latent space
plt.scatter(latent_codes_2d[:, 0], latent_codes_2d[:, 1])
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('Latent Space')
plt.show()
latent_codes_2d = pca.fit_transform(latent_codes)

In [ ]:
test_vector = np.random.normal(size=(1, Z_DIM))
generated_image = decoder.predict(test_vector)

plt.imshow(generated_image[0])
plt.show()

FAUSSES IMAGES

In [ ]:
INPUT_DIM = (128, 128, 3)   # Dimensions des images (hauteur, largeur, canaux)
BATCH_SIZE = 32            # Taille des lots
LATENT_DIM = 200            # Dimension du vecteur latent pour l'autoencodeur
# Z_DIM définit la dimension de l'espace latent
Z_DIM = 200  # Dimension de l'espace latent

# Génération des données par lot avec normalisation des pixels (valeurs entre 0 et 1)
data_flow = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory=DATA_FOLDER,
    classes=['elephant'],
    target_size=INPUT_DIM[:2],  # Redimensionne les images en 128x128
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    class_mode='input'  # Dans un autoencodeur, les entrées et sorties sont identiques
)

# Obtention du premier batch d'images générées
data_batch = next(data_flow)

# Affichage des 30 premières images du batch
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 10, figsize=(12, 6))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(data_batch[0][i])
    ax.axis('off')
plt.show()
print(f"Taille du batch généré : {data_batch[0].shape}")

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AEElephant

epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) (0 si aucun fichier existant) : "))
epochs = int(input("Entrez le numéro d'epochs à faire en plus (ex: 25) : "))
filename = f"AEElephant.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER4, filename)

# === Construction du modèle (toujours reconstruit, puis poids chargés si besoin) ===

# Recréation propre du modèle
encoder_input, encoder_output, shape_before_flattening, encoder = build_encoder(INPUT_DIM, Z_DIM)
decoder_input, decoder_output, decoder = build_decoder(Z_DIM, shape_before_flattening)

autoencoder_input = encoder_input
autoencoder_output = decoder(encoder_output)
autoencoder = Model(autoencoder_input, autoencoder_output)

if epoch_number == 0:
    print("Nouveau modèle créé (pas de poids chargés)")
else:
    if os.path.exists(filepath):
        autoencoder.load_weights(filepath)
        print(f"Poids chargés depuis : {filepath}")
    else:
        print(f"Fichier introuvable : {filepath}")
        sys.exit(1)
initial_epoch = epoch_number
# Compilation de l'autoencodeur avec l'optimiseur Adam et un taux d'apprentissage spécifié,
# ainsi que l'utilisation de la perte 'mean_squared_error' et de l'exactitude comme métrique
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error' , metrics=['accuracy'])

# Création des données de validation pour l'affichage

val_data_flow=tion_data = next(data_flow)

# Définition d'un point de contrôle pour sauvegarder les poids du modèle pendant l'entraînement
# Le modèle sauvegardera uniquement les poids dans un fichier .h5
checkpoint_autoencoder = ModelCheckpoint(os.path.join(WEIGHTS_FOLDER4, 'AEElephant.{epoch:02d}.weights.h5'), save_weights_only = True, verbose=1, save_freq=steps_per_epoch * 25)

# Entraînement de l'autoencodeur avec les données d'entraînement,
# en utilisant les callbacks pour sauvegarder les poids du modèle à chaque époque
history=autoencoder.fit(data_flow,
                     shuffle=True,
                     epochs = initial_epoch+epochs,
                     initial_epoch = initial_epoch,
                     steps_per_epoch=steps_per_epoch,
                     validation_data=val_data_flow,
                     callbacks=[checkpoint_autoencoder])


In [ ]:
INPUT_DIM = (128, 128, 3)   # Dimensions des images (hauteur, largeur, canaux)
BATCH_SIZE = 32            # Taille des lots
LATENT_DIM = 200            # Dimension du vecteur latent pour l'autoencodeur
# Z_DIM définit la dimension de l'espace latent
Z_DIM = 200  # Dimension de l'espace latent

# Génération des données par lot avec normalisation des pixels (valeurs entre 0 et 1)
data_flow = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory=DATA_FOLDER,
    classes=['tiger'],
    target_size=INPUT_DIM[:2],  # Redimensionne les images en 128x128
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    class_mode='input'  # Dans un autoencodeur, les entrées et sorties sont identiques
)

# Obtention du premier batch d'images générées
data_batch = next(data_flow)

# Affichage des 30 premières images du batch
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 10, figsize=(12, 6))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(data_batch[0][i])
    ax.axis('off')
plt.show()
print(f"Taille du batch généré : {data_batch[0].shape}")

In [ ]:
print("vos fichiers disponibles : ")
!ls /content/gdrive/MyDrive/Colab\ Notebooks/ML_FDS/myweights/AETiger

epoch_number = int(input("Entrez le numéro de l'époque à charger (ex: 40) (0 si aucun fichier existant) : "))
epochs = int(input("Entrez le numéro d'epochs à faire en plus (ex: 25) : "))
filename = f"AETiger.{epoch_number:02d}.weights.h5"
filepath = os.path.join(WEIGHTS_FOLDER5, filename)

# === Construction du modèle (toujours reconstruit, puis poids chargés si besoin) ===

# Recréation propre du modèle
encoder_input, encoder_output, shape_before_flattening, encoder = build_encoder(INPUT_DIM, Z_DIM)
decoder_input, decoder_output, decoder = build_decoder(Z_DIM, shape_before_flattening)

autoencoder_input = encoder_input
autoencoder_output = decoder(encoder_output)
autoencoder = Model(autoencoder_input, autoencoder_output)

if epoch_number == 0:
    print("Nouveau modèle créé (pas de poids chargés)")
else:
    if os.path.exists(filepath):
        autoencoder.load_weights(filepath)
        print(f"Poids chargés depuis : {filepath}")
    else:
        print(f"Fichier introuvable : {filepath}")
        sys.exit(1)
initial_epoch = epoch_number
# Compilation de l'autoencodeur avec l'optimiseur Adam et un taux d'apprentissage spécifié,
# ainsi que l'utilisation de la perte 'mean_squared_error' et de l'exactitude comme métrique
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error' , metrics=['accuracy'])

# Création des données de validation pour l'affichage

val_data_flow=tion_data = next(data_flow)

# Définition d'un point de contrôle pour sauvegarder les poids du modèle pendant l'entraînement
# Le modèle sauvegardera uniquement les poids dans un fichier .h5
checkpoint_autoencoder = ModelCheckpoint(os.path.join(WEIGHTS_FOLDER5, 'AETiger.{epoch:02d}.weights.h5'), save_weights_only = True, verbose=1, save_freq=steps_per_epoch * 25)

# Entraînement de l'autoencodeur avec les données d'entraînement,
# en utilisant les callbacks pour sauvegarder les poids du modèle à chaque époque
history=autoencoder.fit(data_flow,
                     shuffle=True,
                     epochs = initial_epoch+epochs,
                     initial_epoch = initial_epoch,
                     steps_per_epoch=steps_per_epoch,
                     validation_data=val_data_flow,
                     callbacks=[checkpoint_autoencoder])


In [ ]:

INPUT_DIM = (128,128,3)
Z_DIM = 200
N_SAMPLES = 5

FOX_DIR = '/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/Data_Project/Tiger-Fox-Elephant/fox'
TIGER_DIR = '/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/Data_Project/Tiger-Fox-Elephant/tiger'

WEIGHTS_FOX = '/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/myweights/AE'
WEIGHTS_NEG = '/content/gdrive/MyDrive/Colab Notebooks/ML_FDS/myweights/AETiger'

def load_n_images(folder, n, target_size):
    files = sorted(os.listdir(folder))[:n]
    imgs = []
    for f in files:
        img = load_img(os.path.join(folder, f), target_size=target_size)
        arr = img_to_array(img) / 255.
        imgs.append(arr)
    return np.stack(imgs, axis=0)

enc_in_f, enc_out_f, shp_f, encoder_fox   = build_encoder(INPUT_DIM, Z_DIM)
dec_in_f, dec_out_f, decoder_fox         = build_decoder(Z_DIM, shp_f)
auto_fox = Model(enc_in_f, decoder_fox(enc_out_f))

ep = int(input("Époque AE fox (0 = none) : "))
if ep>0:
    auto_fox.load_weights(os.path.join(WEIGHTS_FOX, f"AE.{ep:02d}.weights.h5"))
    print("→ fox weights chargés")

enc_in_n, enc_out_n, shp_n, encoder_neg   = build_encoder(INPUT_DIM, Z_DIM)
dec_in_n, dec_out_n, decoder_neg         = build_decoder(Z_DIM, shp_n)
auto_neg = Model(enc_in_n, decoder_neg(enc_out_n))

ep2 = int(input("Époque AE negative (0 = none) : "))
if ep2>0:
    auto_neg.load_weights(os.path.join(WEIGHTS_NEG, f"AETiger.{ep2:02d}.weights.h5"))
    print("→ negative weights chargés")

X_fox = load_n_images(FOX_DIR, N_SAMPLES, INPUT_DIM[:2])
X_neg = load_n_images(TIGER_DIR, N_SAMPLES, INPUT_DIM[:2])

z_fox = encoder_fox.predict(X_fox)
z_neg = encoder_neg.predict(X_neg)

alpha    = 0.5
z_fused  = alpha * z_fox + (1-alpha) * z_neg

X_fused = decoder_fox.predict(z_fused)

fig, axs = plt.subplots(3, N_SAMPLES, figsize=(15,9), dpi=100)
for i in range(N_SAMPLES):
    axs[0,i].imshow(X_fox[i]);      axs[0,i].axis('off')
    axs[1,i].imshow(X_neg[i]);      axs[1,i].axis('off')
    axs[2,i].imshow(X_fused[i]);    axs[2,i].axis('off')

axs[0,0].set_title("Fox originals")
axs[1,0].set_title("Tiger originals")
axs[2,0].set_title("Fusion (50/50)")
plt.tight_layout()
plt.show()


In [ ]:
def generate_images_from_noise(n_to_show=10, Z_DIM=64, decoder=None):
    """
    Génère des images à partir de bruit aléatoire en échantillonnant une distribution normale
    et en utilisant le décodeur d'un modèle pour recréer des images correspondantes.

    Paramètres :
    n_to_show (int, optional): Le nombre d'images à générer et afficher. Défaut à 10.
    Z_DIM (int, optional): La dimension du vecteur de bruit utilisé pour l'échantillonnage. Défaut à 200.
    decoder (model, optional): Le modèle décodeur utilisé pour générer les images à partir du bruit. Il doit être passé en paramètre.
    """

    # Vérification que le modèle décodeur est passé en paramètre
    if decoder is None:
        raise ValueError("Le modèle décodeur n'a pas été fourni. Veuillez passer le modèle décodeur dans le paramètre 'decoder'.")

    # Pour n_to_show images, créer un échantillon suivant une distribution normale
    normal_distribution_sample = np.random.normal(0, 1, size=(n_to_show, Z_DIM))

    # Appliquer la partie décodeur pour générer les images à partir de l'échantillon
    reconst_images = decoder.predict(normal_distribution_sample)

    # Créer une figure pour afficher les images générées
    fig = plt.figure(figsize=(15, 3))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    # Afficher les images générées
    for i in range(n_to_show):
        img = reconst_images[i].squeeze()  # Supprimer la dimension de couleur (s'il y en a)
        sub = fig.add_subplot(2, n_to_show, i + 1)  # Créer un sous-graphe pour chaque image
        sub.axis('off')  # Désactiver les axes pour une meilleure lisibilité
        sub.imshow(img)  # Afficher l'image générée

    # Afficher la figure finale
    plt.show()

In [ ]:
generate_images_from_noise(decoder=decoder)

In [ ]:
# VAE Encoder
def build_vae_encoder(input_dim, output_dim):
    """
    Construit l'encodeur d'un Variational Autoencoder (VAE), qui réduit les images en un espace latent de dimension `output_dim`.

    Paramètres :
    -----------
    input_dim : tuple
        Dimension d'entrée des images (hauteur, largeur, canaux).
    output_dim : int
        Dimension de l'espace latent, ou taille du vecteur latent généré par l'encodeur.

    Retourne :
    --------
    tuple:
        - vae_encoder_input (Input): Le tensor d'entrée pour l'encodeur.
        - vae_encoder_output (Tensor): Le tensor de sortie de l'encodeur après échantillonnage.
        - mean_mu (Tensor): La moyenne de la distribution de l'espace latent.
        - log_var (Tensor): Le logarithme de la variance de l'espace latent.
        - shape_before_flattening (tuple): La forme du tensor avant le Flatten, utilisée pour le décodeur.
        - Model: Le modèle Keras complet de l'encodeur VAE.

    Structure du réseau:
    ---------------------
    1. Couches de convolution pour réduire les dimensions spatiales de l'image tout en augmentant les caractéristiques.
    2. Calcul des statistiques `mean_mu` et `log_var` pour modéliser la distribution de l'espace latent.
    3. La fonction d'échantillonnage applique le "reparameterization trick" pour obtenir un vecteur latent différentiable.
    4. Calcul de la perte KL Divergence pour imposer une régularisation sur l'espace latent.
    """

    # Réinitialiser les sessions précédentes de Keras (utile dans un environnement Jupyter)
    tf.keras.backend.clear_session()

    # Définir l'entrée du modèle encodeur, prenant des images de dimension `input_dim`
    vae_encoder_input = Input(shape=input_dim, name='vae_encoder_input')
    x = vae_encoder_input

    # Première couche de convolution pour extraire les caractéristiques initiales
    x = Conv2D(32, 3, strides=2, padding='same', name='vae_encoder_conv_1')(x)
    x = LeakyReLU()(x)

    # Deuxième couche de convolution pour réduire les dimensions spatiales et augmenter la profondeur
    x = Conv2D(64, 3, strides=2, padding='same', name='vae_encoder_conv_2')(x)
    x = LeakyReLU()(x)

    # Troisième couche de convolution pour extraire davantage de caractéristiques
    x = Conv2D(64, 3, strides=2, padding='same', name='vae_encoder_conv_3')(x)
    x = LeakyReLU()(x)

    # Quatrième couche de convolution pour une réduction spatiale maximale et une extraction de caractéristiques
    x = Conv2D(64, 3, strides=2, padding='same', name='vae_encoder_conv_4')(x)
    x = LeakyReLU()(x)

    # Capture de la forme avant Flatten, utilisée pour le décodeur
    shape_before_flattening = tf.keras.backend.int_shape(x)[1:]
    x = Flatten()(x)

    # Calcul de la moyenne (mu) et du log de la variance (log_var) pour l'échantillonnage dans l'espace latent
    mean_mu = Dense(output_dim, name='mu')(x)
    log_var = Dense(output_dim, name='log_var')(x)

    # Fonction d'échantillonnage appliquant le "reparameterization trick" pour échantillonner de la distribution
    def sampling(args):
        mean_mu, log_var = args
        epsilon = K.random_normal(shape=K.shape(mean_mu), mean=0., stddev=1.)
        return mean_mu + tf.exp(log_var / 2) * epsilon  # Retourne l'échantillon latent

    # Application du sampling avec une couche Lambda
    vae_encoder_output = Lambda(sampling, output_shape=(output_dim,), name='vae_encoder_output')([mean_mu, log_var])

    # Couche de calcul de la perte KL Divergence pour imposer une régularisation de l'espace latent
    class KLLossLayer(tf.keras.layers.Layer):
        def __init__(self, **kwargs):
            super(KLLossLayer, self).__init__(**kwargs)

        def call(self, inputs):
            mean_mu, log_var = inputs
            # Calcul de la divergence KL entre la distribution latente et une normale standard
            kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean_mu) - tf.exp(log_var), axis=1)
            self.add_loss(kl_loss)  # Ajoute la perte KL au modèle
            return kl_loss

    # Initialisation de la couche de perte KL
    kl_loss_layer = KLLossLayer(name='kl_loss')([mean_mu, log_var])

    # Construction du modèle encodeur final du VAE
    vae_encoder = Model(vae_encoder_input, [vae_encoder_output,kl_loss_layer], name='vae_encoder')

    return vae_encoder_input, vae_encoder_output, mean_mu, log_var, shape_before_flattening, vae_encoder


vae_encoder_input, vae_encoder_output, mean_mu, log_var, shape_before_flattening, vae_encoder = build_vae_encoder(INPUT_DIM, Z_DIM)
vae_encoder.summary()

In [ ]:
# DECODEUR

def build_vae_decoder(input_dim, shape_before_flattening):
    """
    Construit le décodeur d'un autoencodeur variationnel (VAE) qui transforme un vecteur latent en une image reconstruite.

    Paramètres:
    -----------
    input_dim : int
        La dimension du vecteur latent, qui est l'entrée du décodeur.
    shape_before_flattening : tuple
        La forme des données avant l'aplatissement dans l'encodeur, utilisée ici pour reconstruire les dimensions
        nécessaires lors de la génération de l'image.

    Retourne :
    --------
    decoder_input : tensorflow.keras.layers.Input
        L'entrée du décodeur, prenant un vecteur de dimension `input_dim`.
    decoder_output : tensorflow.keras.layers.Layer
        La sortie finale du décodeur, une image reconstruite de forme (hauteur, largeur, canaux).
    decoder_model : tensorflow.keras.models.Model
        Le modèle Keras du décodeur, reliant `decoder_input` à `decoder_output`.
    """

    # Définir l'entrée du modèle de décodeur, qui est un vecteur latent de dimension `input_dim`
    decoder_input = Input(shape=(input_dim,), name='decoder_input')

    # Conversion du vecteur en une matrice, en utilisant la forme avant Flatten (shape_before_flattening)
    x = Dense(np.prod(shape_before_flattening))(decoder_input)
    x = Reshape(shape_before_flattening)(x)

    # Première couche Conv2DTranspose pour doubler la dimension spatiale et réduire le nombre de filtres
    x = Conv2DTranspose(filters=64,
                        kernel_size=3,
                        strides=2,
                        padding='same',
                        name='decoder_conv_1')(x)
    x = LeakyReLU()(x)  # Activation pour la reconstruction des détails

    # Deuxième couche Conv2DTranspose pour continuer la reconstruction
    x = Conv2DTranspose(filters=64,
                        kernel_size=3,
                        strides=2,
                        padding='same',
                        name='decoder_conv_2')(x)
    x = LeakyReLU()(x)

    # Troisième couche Conv2DTranspose pour doubler encore la dimension spatiale
    x = Conv2DTranspose(filters=32,
                        kernel_size=3,
                        strides=2,
                        padding='same',
                        name='decoder_conv_3')(x)
    x = LeakyReLU()(x)

    # Quatrième et dernière couche Conv2DTranspose pour ramener les dimensions aux valeurs d'image
    x = Conv2DTranspose(filters=3,
                        kernel_size=3,
                        strides=2,
                        padding='same',
                        name='decoder_conv_4')(x)
    x = Activation('sigmoid')(x)  # Activation sigmoïde pour des valeurs de pixel entre 0 et 1

    # La sortie du décodeur est l'image reconstruite
    decoder_output = x

    # Retourne l'entrée, la sortie, et le modèle complet du décodeur
    return decoder_input, decoder_output, Model(decoder_input, decoder_output)


# Création du décodeur en utilisant la dimension de l'espace latent et la forme avant le flattening
vae_decoder_input, vae_decoder_output, vae_decoder = build_vae_decoder(input_dim = Z_DIM,
                                        shape_before_flattening = shape_before_flattening
                                        )
vae_decoder.summary()

In [ ]:
# Entrée principale du VAE (identique à l'entrée de l'encodeur)
vae_autoencoder_input = vae_encoder_input

# Sortie du VAE en passant par le décodeur, en prenant la sortie de l'encodeur comme entrée
vae_autoencoder_output = vae_decoder(vae_encoder_output)

# Création du modèle complet du VAE avec l'entrée et la sortie spécifiées
vae_autoencoder = Model(vae_autoencoder_input, vae_autoencoder_output)

vae_autoencoder.summary()

In [ ]:
B = 0.0001  # facteur de pondération pour la KL divergence

def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis=[1, 2, 3])  # erreur MSE

def total_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis=[1, 2, 3])  # juste la reconstruction



In [ ]:
DATA_FOLDER = "Data_Project/Tiger-Fox-Elephant/fox"
NUM_IMAGES = len(os.listdir(DATA_FOLDER))  # Nombre d'images total dans le dossier
BATCH_SIZE = 256  # Taille des batchs
IMG_SIZE = (128, 128)  # Taille des images


In [ ]:
# Callback pour sauvegarder les meilleurs poids
checkpoint_callback = ModelCheckpoint(
    filepath="vaeautoencoder_best_weights.weights.h5",  # Nom du fichier pour sauvegarder les meilleurs poids
    save_best_only=True,                        # Sauvegarde uniquement si la perte de validation s'améliore
    monitor='val_loss',                         # Critère de suivi pour la validation
    save_weights_only=True,                     # Sauvegarde des poids
    mode='min',                                 # Mode de suivi (minimiser la perte)
    verbose=1                                   # Affiche un message lors de la sauvegarde
)

# Callback pour arrêter l'entraînement si la perte de validation ne s'améliore pas
early_stopping_callback = EarlyStopping(
    monitor='val_loss',                         # Critère de suivi pour l'arrêt anticipé
    patience=30,                                # Arrête l'entraînement après 10 époques sans amélioration
    mode='min',                                 # Mode de suivi (minimiser la perte)
    restore_best_weights=True,                  # Restaure les meilleurs poids après l'arrêt
    verbose=1                                   # Affiche un message lors de l'arrêt
)

In [ ]:
LEARNING_RATE = 1e-4
EPOCHS = 400
# Définir la perte de reconstruction standard
# Compilation du modèle en intégrant total_loss comme perte
vae_autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=total_loss)


# Calcule le nombre total d'images dans le répertoire spécifié
total_images = len(data_flow.filenames)

# Définir `steps_per_epoch` en divisant le total d'images par le batch_size, en arrondissant vers le haut
steps_per_epoch = int(np.ceil(total_images / BATCH_SIZE))

print(f"Nombre total d'images : {total_images}")
print(f"Taille du batch : {BATCH_SIZE}")
print(f"Nombre de steps par epoch (steps_per_epoch) : {steps_per_epoch}")

# Entraînement du modèle avec les callbacks configurés
history = vae_autoencoder.fit(
    data_flow,
    shuffle=True,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=data_flow,
    callbacks=[checkpoint_callback, early_stopping_callback]  # Ajout des deux callbacks
)

In [ ]:
show_images(vae_autoencoder,example_images)

In [ ]:
visualize_latent_space(encoder_model=vae_autoencoder, images=example_images, num_images=5, image_size=(28, 28))

In [ ]:
def generate_images_from_noise(n_to_show=10):
    """
    Génère et affiche des images à partir d'un bruit aléatoire en utilisant le décodeur du VAE.

    Cette fonction génère des échantillons à partir d'une distribution normale standard (N(0,1)),
    puis utilise le décodeur du Variational Autoencoder (VAE) pour transformer ces échantillons
    en images reconstruites. Les images générées sont ensuite affichées.

    Paramètres :
    n_to_show (int, optionnel): Le nombre d'images à générer et afficher. Par défaut à 10.

    """
    # Génère des échantillons à partir d'une distribution normale (N(0,1)) pour les images à créer
    normal_distribution_sample = np.random.normal(0, 1, size=(n_to_show, Z_DIM))




    # Applique le décodeur pour transformer les échantillons en images
    reconst_images = vae_decoder.predict(normal_distribution_sample)

    # Crée une figure pour afficher les images générées
    fig = plt.figure(figsize=(15, 3))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    # Affiche chaque image générée dans un sous-graphique
    for i in range(n_to_show):
        img = reconst_images[i].squeeze()  # Enlève les dimensions inutiles
        sub = fig.add_subplot(2, n_to_show, i + 1)  # Crée un sous-graphique pour chaque image
        sub.axis('off')  # Désactive les axes autour des images
        sub.imshow(img)  # Affiche l'image

In [ ]:
generate_images_from_noise()

In [ ]:
def generate_images_from_noise(n_to_show=10):
    """
    Génère et affiche des images à partir de bruit aléatoire en utilisant le décodeur du VAE.

    Cette fonction génère des échantillons à partir d'une distribution normale standard (N(0,1)),
    puis utilise le décodeur du Variational Autoencoder (VAE) pour transformer ces échantillons
    en images reconstruites. Les images générées sont ensuite affichées dans une figure avec un
    agencement flexible selon le nombre d'images à afficher.

    Paramètres :
    n_to_show (int, optionnel): Le nombre d'images à générer et afficher. Par défaut à 10.

    """
    # Génère des échantillons à partir d'une distribution normale (N(0,1)) pour les images à créer
    normal_distribution_sample = np.random.normal(0, 1, size=(n_to_show, Z_DIM))

    # Applique le décodeur pour transformer les échantillons en images
    reconst_images = vae_decoder.predict(normal_distribution_sample)

    # Crée une figure pour afficher les images générées
    fig = plt.figure(figsize=(15, 15))  # Augmente la taille de la figure
    rows = int(np.ceil(n_to_show / 4))  # Calcule le nombre de lignes nécessaires
    columns = 4  # Définit le nombre de colonnes par ligne
    fig.subplots_adjust(hspace=0.4, wspace=0.4)  # Ajuste l'espacement entre les images

    # Affiche chaque image générée dans un sous-graphique
    for i in range(n_to_show):
        img = reconst_images[i].squeeze()  # Enlève les dimensions inutiles
        sub = fig.add_subplot(rows, columns, i + 1)  # Positionne l'image dans la grille
        sub.axis('off')  # Désactive les axes autour des images
        sub.imshow(img)  # Affiche l'image


generate_images_from_noise()